# SailGP Data Analyst Challenge

The aim is to test you python abilities. The challenge is to analyze the data provided and answer the questions below. You can use any library you want to help you with the analysis. The data is from the SailGP event in Auckland 2025. The data is in the 'DATA' folder.

There are various sources available.

The Boat Logs are in the 'Boat_Logs' folder. The data is in csv format and the columns are described in the 'Boat_Logs/Boat_Logs_Columns.csv' file.
The 'Course_Marks_2025-01-19.csv' file contains the mark positions and wind reading on the course for the whole day.

The Race_XML folder contains the xml files for each race that contains information on where the boundaries of the course are, the theoretical position of the marks and the target racecourse axis.

The 2025-01-19_man_summary.csv file contains the metrics from the manoeuvre summary for the day.
The 2025-01-19_straight_lines.csv file contains the metrics from the straight line summary for the day.

Both are derived from the boat logs.

The 2502 m8_APW_HSB2_HSRW.kph.csv file contains the polar data for the boats in that config.

## Requierements
- Chose at least 3 questions from the list below to answer.
- Python 3.8 or higher
- Notebook should be able to run without any errors from start to finish.
- Specify the libraries (imports) used in the notebook.
- Any comments to make the notebook self-explanatory and easy to follow would be appreciated.
- If you can't get to the end of a question, we would appreciate the code you have written so far and explain what you were trying to do.

## Further information:
- We usually use bokeh for visualizations. So any showcase of bokeh would be appreciated.
-

## Submitting the results.
It would be great if you could provide a jupyter notebook with the code and the results of the analysis. You can submit the results by sharing a link to a git repository.


### Imports and re-used functions
Free section to initialize the notebook with the necessary imports and functions that will be used in the notebook.



In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import math


''' Choose a race and boat to analyze '''

race_number = 25011905  
boat = 'SUI'


# Import the log files
log_file_paths = [
    'Data/Boat_logs/data_AUS.csv',
    'Data/Boat_logs/data_BRA.csv',
    'Data/Boat_logs/data_CAN.csv',
    'Data/Boat_logs/data_DEN.csv',
    'Data/Boat_logs/data_ESP.csv',
    'Data/Boat_logs/data_GBR.csv',
    'Data/Boat_logs/data_GER.csv',
    'Data/Boat_logs/data_ITA.csv',
    'Data/Boat_logs/data_NZL.csv',
    'Data/Boat_logs/data_SUI.csv',
    'Data/Boat_logs/data_USA.csv',
]

log_dataframes = {}     # Dictionary of log files
for file in log_file_paths:
    country = file.replace("Data/Boat_logs/data_", "").replace(".csv", "")  # Extract the country code
    df = pd.read_csv(file)   # Read the file
    log_dataframes[country] = df    # Store the logs for each country


# Load XML File
xml_file_path = "Data/Race_XMLs/25011905_03-13-55.xml"
with open(xml_file_path, "r") as f:
    soup = BeautifulSoup(f, "lxml-xml")  # Use lxml parser

# Parse Course XML
def parse_course_xml(soup):
    """Parses the course XML using BeautifulSoup to extract mark locations and sequence."""
    course_marks = []
    for compound_mark in soup.find_all("CompoundMark"):  # Find all compound marks
        for mark in compound_mark.find_all("Mark"):  # Find all individual marks within compound marks
            course_marks.append({
                'id': int(compound_mark["CompoundMarkID"]),  # Extract the compound mark ID, name, lat and lon
                'name': mark["Name"], 
                'lat': float(mark["TargetLat"]),  
                'lon': float(mark["TargetLng"])  
            })
    
    # Extract the ordered sequence of marks based on the course rounding sequence
    sequence = [int(corner["CompoundMarkID"]) for corner in soup.find_all("Corner")]
    
    return course_marks, sequence

marks, sequence = parse_course_xml(soup)


# Distance between two points on a round Earth

def distance_between_points(lat1, lon1, lat2, lon2):
    
    R = 6371000  # Earth radius 
    phi1, phi2 = np.radians(lat1), np.radians(lat2) # Convert to radians
    delta_phi = np.radians(lat2 - lat1) # Calculate the change in latitude
    delta_lambda = np.radians(lon2 - lon1)  # Calculate the change in longitude
    
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2  # Haversine formula
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)) # Calculate the great circle distance
    return R * c  # Distance in meters


# Distance to the next mark 

def calculate_distance_to_next_mark(log_dataframes, race_number):
    for country, df in log_dataframes.items():
        race_df = df[df['TRK_RACE_NUM_unk'] == race_number].copy() # Filter the dataframe for the specific race number
        race_df['distance_to_next_mark'] = np.nan  # Initialize the column 
        
        for i in race_df.index:  # Iterate through each column in the dataframe
            lat, lon, leg = race_df.loc[i, ['LATITUDE_GPS_unk', 'LONGITUDE_GPS_unk', 'TRK_LEG_NUM_unk']]
            leg = int(leg)  # Convert leg number to integer
            
            if leg < len(sequence):  # Ensure within valid range
                target_mark_id = sequence[leg]  # Get the next mark in the sequence
                target_mark = next(m for m in marks if m['id'] == target_mark_id)  # Get correct mark from list
                distance_to_mark = distance_between_points(lat, lon, target_mark['lat'], target_mark['lon'])  # Calculate distance
            else:
                distance_to_mark = 0  # If the boat has finished, set distance to 0
            
            race_df.at[i, 'distance_to_next_mark'] = distance_to_mark  # Store distance 
        
        log_dataframes[country] = race_df  # Update 

# Function to sync different boat data with the start time - apply once you've extracted the data for one race
def synchronize_boat_data(df):
    start_index = df[df['PC_TTS_s'] < 0].index.min() # Time to start goes from positive to negative
    return df


## Question 1: Write a Python function that can take a compass direction (ie. TWD or Heading) and calculate an accurate mean value across a downsampled frequency. Eg. If TWD is at 1Hz, give me a 10s average.

In [55]:

# Function to calculate circular mean for compass directions
def circular_mean(angles):
    angles_rad = np.deg2rad(angles.dropna())  # Convert to radians
    sin_mean = np.mean(np.sin(angles_rad))
    cos_mean = np.mean(np.cos(angles_rad))
    mean_angle_rad = np.arctan2(sin_mean, cos_mean)  # Compute mean in radians
    mean_angle_deg = np.rad2deg(mean_angle_rad)  # Convert back to degrees
    return mean_angle_deg % 360  # Make sure the result is within 0 and 360

# Step 3: Function to downsample data
def downsample(df, time_column, direction_column):
    df[time_column] = pd.to_datetime(df[time_column])  # Make sure time is datetime
    df.set_index(time_column, inplace=True)  # Set time as index

    downsampled_df = df.resample('10S').apply({direction_column: circular_mean}) # Resample to 10 second intervals and calculate circular mean
    
    return downsampled_df.reset_index()  # Reset index to get time back as a column


# Example Usage
# Call a specific dataset by country
df_downsampled = downsample(log_dataframes[boat], time_column='TIME_LOCAL_unk', direction_column='HEADING_deg')
print(df_downsampled.head(10))

       TIME_LOCAL_unk  HEADING_deg
0 2025-01-19 16:06:00    37.006060
1 2025-01-19 16:06:10    60.914908
2 2025-01-19 16:06:20    70.824615
3 2025-01-19 16:06:30    77.440969
4 2025-01-19 16:06:40    87.544429
5 2025-01-19 16:06:50    97.262034
6 2025-01-19 16:07:00   114.610449
7 2025-01-19 16:07:10   116.896025
8 2025-01-19 16:07:20   117.085006
9 2025-01-19 16:07:30    63.519013


C:\Users\fredd\AppData\Local\Temp\ipykernel_58480\685738536.py:15: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  downsampled_df = df.resample('10S').apply({direction_column: circular_mean}) # Resample to 10 second intervals and calculate circular mean


## Question 2: Given a course XML and a timeseries of boat Lat/Lon values, calculate a VMC column for the same timeseries.


In [56]:


# Function to calculate the VMC for each timestep

def calculate_vmc(df, marks, sequence, race_number):
    df.loc[df['TRK_RACE_NUM_unk'] == race_number, 'vmc'] = 0.0  # Initialize VMC column only for the selected race
    
    race_df = df[df['TRK_RACE_NUM_unk'] == race_number] # Filter out this race number
    
    for i in race_df.index:
        lat, lon, heading, speed, leg = df.loc[i, ['LATITUDE_GPS_unk', 'LONGITUDE_GPS_unk', 'GPS_COG_deg', 'GPS_SOG_km_h_1', 'TRK_LEG_NUM_unk']]
        leg = int(leg)  # Convert leg number to integer
        
        if leg - 1 < len(sequence):  # Ensure within valid range
            target_mark_id = sequence[leg - 1]  # Get the next mark in the sequence
            target_mark = next(m for m in marks if m['id'] == target_mark_id)  # Get correct mark from list
            
            lat1, lon1, lat2, lon2 = map(math.radians, [lat, lon, target_mark['lat'], target_mark['lon']]) # Convert to radians 
            delta_lon = lon2 - lon1
            
            x = math.sin(delta_lon) * math.cos(lat2) # Compute bearing between the two points
            y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)) 
            bearing = math.atan2(x, y)
            
            mark_vector = np.array([math.sin(bearing), math.cos(bearing)]) # Calculate a mark vector, the direction to the mark

            boat_heading_rad = np.radians(heading)  # Convert heading from degrees to radians
            boat_vector = np.array([np.sin(boat_heading_rad), np.cos(boat_heading_rad)]) # Calculate the boat vector
        
            vmc = speed * np.dot(boat_vector, -mark_vector) # Calculate VMC as the projection of boat velocity onto the direction to the mark
            df.at[i, 'vmc'] = vmc  # Store VMC value

    return df


log_dataframes[boat] = calculate_vmc(log_dataframes[boat], marks, sequence, race_number)  # Compute VMC for the race for SUI


print(log_dataframes[boat]['vmc'].head(10))  


TIME_LOCAL_unk
2025-01-19 16:06:01     6.051192
2025-01-19 16:06:02     6.497582
2025-01-19 16:06:03     7.800497
2025-01-19 16:06:04     8.903854
2025-01-19 16:06:05     8.765837
2025-01-19 16:06:06     9.732845
2025-01-19 16:06:07    10.049996
2025-01-19 16:06:08     9.545832
2025-01-19 16:06:09     9.460450
2025-01-19 16:06:10     9.415283
Name: vmc, dtype: float64


## Question 3: Verify and comment on the boats calibration. If possible propose a post-calibrated set of wind numbers and a potential calibration table.


There are many ways to go about this section but we will start with the straight line data boat speeds, compass headings and the GPS COG and SOG to verify the calibration of these instruments. Checking the tidal info and assuming there is little current, we should see if there are any disparities between the two and check if they can be attributed to different heel angles (can mess with compass and paddle wheels), boat speeds (can change with paddle wheels), tacks, and if the differences are consistent between boats. 

Now that we have an accurate idea of the heading and speed, we can begin to adress the wind triangle. We need to know if the angles provided by the wind gear are based off the compass heading or the GPS COG to be able to single out calibration errors in the wind gear.

Sorting tacks and gybes into different bins based on the wind speed, we can compare the evolution of the TWD, TWA and AWA from tack to tack. Since the former two are calculated from the AWA, these should be the same between tacks if the calibration is right and the boats are sailing the same angles on each tack. 
We can also calculate the TWD based on the mean headings before and after tacks and gybes to see if there is any disparity between the change in heading, and the sum of the two TWAs. 

For TWS, we can sort the up and downwind TWS readings looking at the top mark roundings in the log files, or straight line data that are close in time. If there is a strong disparity between TWS numbers, there can be some calibration issues to adress. We can calculate our own AWA and AWS arrays based on the instruments TWS, TWA and the COG and SOG, and we can compare these to the instruments measured raw AWA and AWS values. The differences will reveal the pre-existing calibration, and whether it should change. An advantage or having other boats and instruments on the marks is that we can observe the differences between them. 

To adress some calibration errors brought on by directional wind shear, we can check if there are any trends in the sail twist data that reveal differences in trim between tacks. If the sailors are good and there are no other reasons (offset tide, chop, dirty air, sailing a course to avoid an island or boat rather than sailing best VMG), then there could be some differences in twist that could result in errors in TWD. 

With my dissertation for my masters in meteorology, I have scripts that can estimate the wind shear based on meteorological factors and can help understand the differences between the wind speed at the top of the mast, the marks and the 10 m forecast. 

Finally, we can adjust the frequencies of the data and filter it to remove outliers. We can also check the VMG data to see any patterns between bad calibration and over/ underperformance on different tacks.


## Question 4: Given a timeseries of Lat/Lon positions and a course XML, in a Python notebook, calculate a Distance to Leader metric for each boat.

In [57]:
# Calculation of distance to finish, uses distance to next mark and adds up the distances between the other ones to the finish

def calculate_distance_to_finish(log_dataframes, race_number):
    
    for country, df in log_dataframes.items(): # Do it for all the countries since we need a leaderboard
        race_df = df[df['TRK_RACE_NUM_unk'] == race_number].copy() # Take out a race number
        distances = [] # Initialize
        for _, row in race_df.iterrows():
            lat, lon, leg = row['LATITUDE_GPS_unk'], row['LONGITUDE_GPS_unk'], int(row['TRK_LEG_NUM_unk'])
            total_distance = 0  # Initialize 
            
            for next_leg in range(leg, len(sequence)): # For all the legs
                next_mark = next(m for m in marks if m['id'] == sequence[next_leg]) # Get the next mark
                if next_leg == leg:  # First leg from current position
                    total_distance += distance_between_points(lat, lon, next_mark['lat'], next_mark['lon']) # Calculate distance to the next mark
                else:  # For all the other legs
                    prev_mark = next(m for m in marks if m['id'] == sequence[next_leg - 1])
                    total_distance += distance_between_points(prev_mark['lat'], prev_mark['lon'], next_mark['lat'], next_mark['lon']) # Calculate the distance between the other marks 
            
            distances.append(total_distance)
        
        race_df['distance_to_finish'] = distances
        log_dataframes[country] = race_df  # Update the original data structure
    
    return log_dataframes

# Function to calculate the distance to leader

def rank_and_distance_to_leader(log_dataframes, race_number):
    
    log_dataframes = calculate_distance_to_finish(log_dataframes, race_number)  # Calculate the distance to finish

    for country, df in log_dataframes.items():
        race_df = df[df['TRK_RACE_NUM_unk'] == race_number].copy() # Get the data for one race
        race_df = synchronize_boat_data(race_df) # Sync the boats based on the start
        race_df['distance_to_leader'] = np.nan  # Initialize 

        for time in race_df['DATETIME'].unique():  # For each timestep 
            snapshot = {country: df[df['DATETIME'] == time] for country, df in log_dataframes.items()} # Make snapshots of data
            snapshot = {k: v for k, v in snapshot.items() if not v.empty}  # Remove empty snapshots

            if snapshot:  # If there
                ranked_boats = sorted(snapshot.items(), key=lambda x: x[1]['distance_to_finish'].iloc[0]) #  Rank boats by distance to finish
                leader_distance = ranked_boats[0][1]['distance_to_finish'].iloc[0]  # Get leader's distance

                for rank, (country, boat_data) in enumerate(ranked_boats): # For each boat and ranking
                    if not boat_data.empty:
                        distance_to_leader = boat_data['distance_to_finish'].iloc[0] - leader_distance # Calculate distance to leader
                        log_dataframes[country].loc[boat_data.index, 'distance_to_leader'] = distance_to_leader # Store distance

    return log_dataframes


log_dataframes = rank_and_distance_to_leader(log_dataframes, race_number)  # Compute DTL


print(log_dataframes[boat]['distance_to_leader'].head(100))  

TIME_LOCAL_unk
2025-01-19 16:06:01    216.397515
2025-01-19 16:06:02    220.446037
2025-01-19 16:06:03    223.268016
2025-01-19 16:06:04    224.292865
2025-01-19 16:06:05    223.574710
                          ...    
2025-01-19 16:07:36    133.751510
2025-01-19 16:07:37    131.317545
2025-01-19 16:07:38    129.719884
2025-01-19 16:07:39    129.407567
2025-01-19 16:07:40    129.999727
Name: distance_to_leader, Length: 100, dtype: float64


## Question 5: Given a course XML, along with a wind speed and direction and a polar, calculate the minimum number of tacks or gybes for each leg of the course and each gate mark on the leg.

In [58]:
'''

Function:
        def calculate_new_coordinates
            Calculate new coodinates based on the speed and heading for 10 seconds forwards
            If the new coordinates are within the boundary coordinates:
            continue
            If the new coordinates aren't:
            If we are on port tack add 90 degrees to the heading, recalculate position
            If we are on starboard, take away 90 degrees (assuming we tack and gybe through the same angle)
            Count +1 tack/gybe (can seperate tack or gybe)
            Calculate a new TWA range
        return the new coordinates, the tack/gybe counter, and the TWA range



To make this more simple, we can choose a race number and take an average wind speed and direction from the averages of all the marks data. We can then establish which start mark is closest to the first mark and start from these coordinates.

Define the initial tack to create a TWA range (0-180 or 180 to 360), make the polar TWA range from 0-360, define the marks list, initialise all the apropriate arrays

For each mark:

    Calculate the TWA within the current TWA range with the best VMC towards the mark based on the TWS, TWD and mark angle calculated between our current coordinates and the mark coords.

    Extract the BSP from the polar and calculate the heading based on TWA and TWD. 

        Use the calculate_new_coordinates function
        If new coordinates are within a certain range of a mark lets say 30m, go to next mark, else repeat the calculate new coordinates function.
        add +10 seconds to timer array

This should give us an array of time, lat, lon, twa, bsp, heading and tack and gybe counter.

'''

"\n\nFunction:\n        def calculate_new_coordinates\n            Calculate new coodinates based on the speed and heading for 10 seconds forwards\n            If the new coordinates are within the boundary coordinates:\n            continue\n            If the new coordinates aren't:\n            If we are on port tack add 90 degrees to the heading, recalculate position\n            If we are on starboard, take away 90 degrees (assuming we tack and gybe through the same angle)\n            Count +1 tack/gybe (can seperate tack or gybe)\n            Calculate a new TWA range\n        return the new coordinates, the tack/gybe counter, and the TWA range\n\n\n\nTo make this more simple, we can choose a race number and take an average wind speed and direction from the averages of all the marks data. We can then establish which start mark is closest to the first mark and start from these coordinates.\n\nDefine the initial tack to create a TWA range (0-180 or 180 to 360), make the polar TWA 

## Question 6: Calculate a “tacked” set of variables depending on the tack of the boat, so that sailors don’t need to think about what tack they’re on when looking at measurements. And show the results in a visualisation.


For this section, I would make number boxes that show the different values. The sailors may want different variables on their screens but the ones that need "tacking" mathematically are true and apparent wind angles, heel angles, rudder angle, and foil information. Rather than port and starboard information we make an active and inactive array. We can also make course information with distances, angles, laylines times and distances. These can also change colour and be flipped, between the closest layline and the layline on the other tack - my preference for it to be read first. We can also make some "what if?" boxes with the angles on the other tack or gybe. The frequencies of the data coming on the screen also needs some damping for certain arrays. Significant figures can also be adjusted. We can use the polars to calculate targets to display.

## Question 7: Given a set of tacks (in CSV), and train a model to explain the key features of these tacks when optimizing for vmg. Show appropriate visualisations to explain your conclusions.

This would only use the manoeuvre data provided. Starting by filtering out just the tacks and filling blanks with nan values. We can also drop some irrelevant arrays like the gforce. We can also merge the specific port and starboard data for foil drops for example. 

We can establish what data can be used for testing the model. We have VMG distances along with predictions and target, so establishing the correct arrays and calculating the deltas between predicted and real VMGs, as well as comparing these to the "loss" columns are important for understanding what to test against. Some clarification about the calculations behind these arrays would be useful. 

Then we can train a model to go through each array and establish what constitutes a good or bad tack. We can also visualise data in different ways by sorting the best and wost 10%, 25% or 50% of tacks. 

This process should indicate the features of a good and bad tack and we can also compare the boats and time since we have these arrays to sort the data. 

## Question 8: Give insights on the racing on what made a team win or underperform in the race.

To decide what made a team win or underperform in a race is very broad, particularly in SailGP with such chaotic racing. It can be useful to watch the races back to see where these exceptional moments happened. The straight line data of a boat might be fantastic, but if they have a crash or had to duck 5 boats at the leeward gate, the race will be compromised.

We can start with the starts and order at the first mark. The time, distance and speed to the line can make or break the race, and can have its own analysis. 

We can also seperate the tacks and gybes performance of each team. The number of tacks and gybes is important too since they cost a lot for these fast boats.

Mark roundings are also important with gates being a hot spot for position changes. We can monitor the leaderboard before and after each mark and examine the course to find out which mark was the best for VMC. A loss of position and the wrong mark can lose a race, and a crafty rounding of the right mark and win a race. 

We can now have a look at the average data for the whole race and legs, VMG, VMC, boat and wind speeds, performance on targets...

For this section, we can use functions from most of the previous questions. 
